In [2]:
import pandas as pd

In [3]:
import numpy as np

### Pre processing - Cleaning the dataset

Obtaining tags from the given dataset of Search terms

In [4]:
Travel = pd.read_csv("Travel.csv")

In [5]:
two_wheelers = pd.read_csv("2W.csv")

In [6]:
four_wheelers = pd.read_csv("4W.csv")

In [7]:
brand_terms = pd.read_csv("BrandTerms.csv")

In [8]:
Health = pd.read_csv("Health.csv")

In [9]:
Travel

Terms
0                                   travel insurance
1                          overseas travel insurance
2                              visa travel insurance
3                            schengen visa insurance
4                                     visa insurance
..                                               ...
116           "bajaj allianz travel insurance india"
117                  "bajaj travel insurance policy"
118  "bajaj allianz overseas travel insurance india"
119          "bajaj allianz travel health insurance"
120           "bajaj allianz travel insurance rates"

[121 rows x 1 columns]

In [10]:
def removeSplChars(test_string):
    bad_chars = [';', ':', '!', "*", '"',"[","]","+"]
    act_str = test_string
    for i in bad_chars :
        act_str = act_str.replace(i, '')
    return str(act_str)

In [11]:
def MakeArray(col):
    Arr_col = col.to_numpy()
    Arr = []
    for i in Arr_col:
        q = removeSplChars(i)
        Arr.append(q)
    Arr = np.array(Arr)
    
    # Refining
    Arr_cleaned = []
    for word in Arr:
        p = [str(word) for word in word.split()]
        for i in p:
            Arr_cleaned.append(i)
    
    # Removing Duplicates
    Arr_cleaned2 = set(Arr_cleaned)
    Arr_cleaned2 = list(Arr_cleaned2)
    
    # Converting data into lower
    for i in range(len(Arr_cleaned2)):
        Arr_cleaned2[i] = Arr_cleaned2[i].lower()
    
    return Arr_cleaned2

In [12]:
Travel_cleaned = MakeArray(Travel["Terms"])
two_wheelers_cleaned = MakeArray(two_wheelers["Terms"])
four_wheelers_cleaned = MakeArray(four_wheelers["Terms"])
brand_terms_cleaned = MakeArray(brand_terms["Terms"])
Health_cleaned = MakeArray(Health["Terms"])

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mailm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def RemoveStopWords(col):
    stop_words = set(stopwords.words('english'))
    filtered = []
    for i in col:
        if i in stop_words:
            continue
        filtered.append(i)
    return filtered

In [15]:
Travel_cleaned2 = RemoveStopWords(Travel_cleaned)
two_wheelers_cleaned2 = RemoveStopWords(two_wheelers_cleaned)
four_wheelers_cleaned2 = RemoveStopWords(four_wheelers_cleaned)
brand_terms_cleaned2 = RemoveStopWords(brand_terms_cleaned)
Health_cleaned2 = RemoveStopWords(Health_cleaned)

In [16]:
data2 = pd.concat([pd.Series(Travel_cleaned2,name='Travel'),pd.Series(two_wheelers_cleaned2,name='2W'),pd.Series(four_wheelers_cleaned2,name='4W'),pd.Series(brand_terms_cleaned2,name='Brand Terms'),pd.Series(Health_cleaned2,name='2W')], axis=1)

In [17]:
data2

Travel          2W          4W Brand Terms             2W
0    insure  comparison      review      review     comparison
1        65         150  comparison  comparison      mediclaim
2      long       bajaj   temporary       total            top
3      cost        year         top         top      providers
4     india        plus   providers    policies       existing
..      ...         ...         ...         ...            ...
219     NaN         NaN         NaN         NaN        waiting
220     NaN         NaN         NaN         NaN        cardiac
221     NaN         NaN         NaN         NaN      insurence
222     NaN         NaN         NaN         NaN  international
223     NaN         NaN         NaN         NaN           bupa

[224 rows x 5 columns]

In [18]:
def recommend(word):
    flag = False
    counter = 0
    if word in Travel_cleaned2:
        flag = True
        counter = 1
#         print("Travel Insurance recommendations")
    if word in two_wheelers_cleaned2:
        flag = True
        counter = 2
#         print("Two Wheelers Insurance recommendations")
    if word in four_wheelers_cleaned2:
        flag = True
        counter = 3
#         print("Four Wheelers Insurance recommendations")
    if word in brand_terms_cleaned2:
        flag = True
        counter = 4
#         print("Brand Insurance recommendations")
    if word in Health_cleaned2:
        flag = True
        counter = 5
#         print("Health Insurance recommendations")
        
#     if flag == False:
#         print("We didn't understand you, but here are the trending results in your Geography location")
    return (flag,counter)

In [19]:
# recommend("vacation")

In [20]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [21]:
def Similarity(T,w):
    for i in T:
        if similar(i,w) >= 0.8:
            return True
    return False

In [22]:
BroadLabels = {
    'Travel' : 1,
    'Two Wheelers' : 2,
    'Four Wheelers' : 3,
    'Brand Insurance' :4,
    'Health Insurance' : 5
}

In [23]:
def PrimaryFiltering(word):
#     A = recommend(word)
#     fl = A[0]
#     cn = A[1]
    fl = False
    cn = 0
#     print(f"Bool : {fl}, count : {cn}\n")        
    
    # When flag is false, Similarity based matching of keys is done
    lst = [] #list of relevant recommendations
    if Similarity(Travel_cleaned2,word):
#             print("Travel Insurance recommendations")
        cn = 1
        lst.append(cn)
        fl = True
    if Similarity(two_wheelers_cleaned2,word):
#             print("Two Wheelers Insurance recommendations")
        cn = 2
        lst.append(cn)
        fl = True
    if Similarity(four_wheelers_cleaned2,word):
#             print("Four Wheelers Insurance recommendations")
        cn = 3
        lst.append(cn)
        fl = True
    if Similarity(brand_terms_cleaned2,word):
#             print("Brand Insurance recommendations")
        cn = 4
        lst.append(cn)
        fl = True
    if Similarity(Health_cleaned2,word):
#             print("Health Insurance recommendations")
        cn = 5
        lst.append(cn)
        fl = True
        
#     print(f"Primary : {lst} and Bool : {fl}\n")        
    return (lst,fl) # Return whether primary filter is success or failure           

In [24]:
PrimaryFiltering("bajaj")

([1, 2, 3, 4, 5], True)

In [25]:
TravelSpecific = pd.read_csv("Act_Travel.csv")
TwoWSpecific = pd.read_csv("Act_2W.csv")
FourWSpecific = pd.read_csv("Act_4W.csv")
HealthSpecific = pd.read_csv("Act_Health.csv")

In [26]:
for i in TravelSpecific.columns:
    TravelSpecific[i].str.lower()
    
for i in TwoWSpecific.columns:
    TwoWSpecific[i].str.lower()
    
for i in FourWSpecific.columns:
    FourWSpecific[i].str.lower()
    
for i in HealthSpecific.columns:
    HealthSpecific[i].str.lower()

In [27]:
# Making a huge list of dictionaries
TSDict = {}
for i in TravelSpecific.columns:
    a = list(TravelSpecific[i].dropna())
    TSDict[i] = a

TwoSDict = {}
for i in TwoWSpecific.columns:
    a = list(TwoWSpecific[i].dropna())
    TwoSDict[i] = a
    
FSDict = {}
for i in FourWSpecific.columns:
    a = list(FourWSpecific[i].dropna())
    FSDict[i] = a
    
HSDict = {}
for i in HealthSpecific.columns:
    a = list(HealthSpecific[i].dropna())
    HSDict[i] = a
    

In [28]:
def SecondaryFiltering(word):
    res = []
    flg = False
    
    for key in TSDict:
        if word in TSDict[key]:
            res.append(key)
            flg = True
            
    for key in TwoSDict:
        if word in TwoSDict[key]:
            res.append(key)
            flg = True
            
    for key in FSDict:
        if word in FSDict[key]:
            res.append(key)
            flg = True
            
    for key in HSDict:
        if word in HSDict[key]:
            res.append(key)
            flg = True
    return (res,flg)

In [29]:
Links = pd.read_csv("InsuranceLinks.csv")

In [30]:
Links.set_index('Insurance Name', inplace=True) #Setting index as insurance names

In [31]:
Links['counter'] = 0 # Adding a counter column to find the frequency

In [32]:
def IncrementCounter(Name):
    if Links["Website"][Name]:
        cnt = Links['counter'][Name] 
        cnt = cnt + 1 # Increment
        Links.at[Name,'counter'] = cnt

In [33]:
Connection = pd.read_csv("BroadNSpecs.csv")
ColNames2 = Connection.columns
Conn_Dict = {}
for i in ColNames2:
    a = list(Connection[i].dropna())
    Conn_Dict[i] = a

In [34]:
def FindKey(tag):
#     res2 = []
    for key in Conn_Dict:
        if tag in Conn_Dict[key]:
#             res2.append(key)
            return key

In [35]:
def SpecificSuggestion(l):
    for i in l:
        print(f"{i} --> {Links['Website'][i]}")

In [36]:
def FindCounter(Col):
    return Links['counter'][Col] 

In [37]:
def SortIndex(TotLinks):
    # Sort the 'TotLinks on rank'
        CounterVal = []
        # Getting counter values
        for i in TotLinks:
            ind = FindCounter(i)
            CounterVal.append(ind)
        # Sorting TotLinks wrt CounterVal
        SortedLinks = [x for _,x in sorted(zip(CounterVal,TotLinks), reverse = True)]
#         SortedLinks = [x for _,x in sorted(zip(TotLinks,CounterVal))]
#         print(SortedLinks)
        return SortedLinks

In [38]:
def CalcTrends():
    Complete_Links = list(Links.index)
    Trends_Links = SortIndex(Complete_Links)
    return Trends_Links

In [39]:
IncrementCounter("Asia Travel Insurance")

In [40]:
IncrementCounter("Travel Ace Plan")

In [41]:
def FindBroadIndex(tag):
    for key in BroadLabels:
        if BroadLabels[key] == tag:
            return key

In [42]:
# Editing
def Case3(Lst): # PFF = False and SFF = True -> go specific + back track    
    primary_index = Lst[0]
    secondary_index = Lst[1]
    ActLinks = [] # The actual ones
    TotLinks = [] # The excess ones apart from the ones we know
    Keys = [] # Whatever keys we've got
    
    # Adding Actual Links
    for i in secondary_index:
        if i != None:
            ActLinks.append(i)
            
            
    if ActLinks == []:
        # Adding Actual Links
        r = []
        for i in primary_index:
            if i != None:
                x = FindBroadIndex(i)
                r.append(x)
        for i in r:
            z = FindKey(i)
            Keys.append(z)
            
        for j in Keys:
            ActLinks.append(Conn_Dict[i])
            
            
        
    ActResult = SortIndex(ActLinks)
    
    if secondary_index != []:
        if len(secondary_index) > 1:
            for i in range(len(secondary_index)):
                q = FindKey(secondary_index[i])
                Keys.append(q)

            # Add corresponding sub-headings
            for i in Keys:
                a = Conn_Dict[i]
                for j in a:
                    TotLinks.append(j)

        elif len(secondary_index) == 1:
            Keys = FindKey(secondary_index)
#             if (Conn_Dict[Keys]) != None:
            if Keys != None:
                TotLinks = Conn_Dict[Keys]
            else:
                TotLinks = CalcTrends()
            
#             for key in BroadLabels:
#                 if BroadLabels[key] == primary_index:
#                     Keys = key
#                     TotLinks = Conn_Dict[key]
#                     break

#         print(f"before : {len(TotLinks)}")
        TotLinks = set(TotLinks)
#         print(f"after : {len(TotLinks)}")
        Result = SortIndex(TotLinks)
        
    elif primary_index != []:
        if len(primary_index) > 1:
            # get the broad keys
            for i in range(len(primary_index)):
#                 Keys.append(BroadLabels[primary_index])
                for key in BroadLabels:
                    if BroadLabels[key] == primary_index:
                        Keys.append(key)
                        TotLinks.append(Conn_Dict[key])
                    else:
                        return False
                    
                
            # get specific keys
            for i in Keys:
                a = Conn_Dict[i]
                for j in a:
                    TotLinks.append(j)
            
        elif len(primary_index) == 1:
            for key in BroadLabels:
                if BroadLabels[key] == primary_index:
                    Keys = key
                    TotLinks = Conn_Dict[key]
#                     TotLinks = 
                    if Keys == None:
                        TotLinks = CalcTrends()
                    
#             Keys = BroadLabels[primary_index]
#             TotLinks = Conn_Dict[Keys]

            
#         print(f"before : {len(TotLinks)}")
        TotLinks = set(TotLinks)
#         print(f"after : {len(TotLinks)}")
        Result = SortIndex(TotLinks)
        
    else:
        ActResult = CalcTrends()
    return (ActResult,Result)
        

In [43]:
def Recommendation_Engine(word): # Complete recommender system
    
    
    (First_ans,PFF) = PrimaryFiltering(word) # Primary Filtering Flag
    (Sec_ans,SFF) = SecondaryFiltering(word) # Secondary Filtering Flase
    
    if word in brand_terms_cleaned2:
        PN = CalcTrends()
    elif (PFF == True) and (SFF == True):
        PN = Case3((First_ans,Sec_ans))
        print("Case 1")
    elif (PFF == True) and (SFF == False):
        print(First_ans,Sec_ans)
        PN = Case3((First_ans,Sec_ans))
        print("Case 2")
    elif (PFF == False) and (SFF == True):
        PN = Case3((First_ans,Sec_ans))
        print("Case 3")
    elif (PFF == False) and (SFF == False):
        PN = Case3((First_ans,Sec_ans))
    else:
        PN = CalcTrends()
        print("Case 4")
    return PN

In [44]:
def Input():
    search_query = input("Enter your query : ")
    
    # Pre-Processing
    filtered_query = removeSplChars(search_query)
    print(filtered_query)
    word_cleaned = filtered_query.split()
    for i in range(len(word_cleaned)):
        word_cleaned[i] = word_cleaned[i].lower()
    filtered_query = word_cleaned
    word_cleaned = RemoveStopWords(filtered_query)
    word_cleaned = set(word_cleaned)
    word_cleaned = list(word_cleaned)
#     Recommendation_Engine(word_cleaned[1])
    a = []
    for i in word_cleaned:
        a.append(Recommendation_Engine(i))
    print(a)
    print(word_cleaned)

In [46]:
Input()

Enter your query : school
school
Case 3
[(['Student Travel Insurance'], ['Travel Ace Plan', 'Asia Travel Insurance', 'Two Wheeler Third Party Insurance', 'Two Wheeler Insurance', 'Top Up Health Insurance', 'Third Party Car Insurance', 'Student Travel Insurance', 'Senior Citizen Travel Insurance', 'Senior Citizen Health Insurance', 'Saral Suraksha Bima', 'Property Insurance', 'Pet Dog Insurance', 'My Home Insurance', 'Marine Insurance', 'M-Care', 'Long Term Two Wheeler Insurance', 'Liability Insurance', 'Individual Travel Insurance', 'Individual Health Insurance', 'Individual Cyber Insurance', 'Householders Policy', 'Health Insurance for Coronavirus', 'Health Infinity Plan', 'Group Mediclaim Policy', 'Global Personal Guard', 'Financial Lines Insurance', 'Family Travel Insurance', 'Family Health Insurance', 'Extended Warranty Insurance', 'Engineering Insurance', 'Domestic Travel Insurance', 'Critical Illness Insurance', 'Criti Care Policy', 'Corporate Travel Insurance', 'Corona Kavach Po